In [2]:
import pandas as pd
import requests
import time
import re
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

# Logging in Instagram

In [123]:
def instagram_login(driver, username, password):
        """
        Returns driver logged in to instagram
        """

        # Login url
        driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')

        # Wait 3 seconds to make instagram think I'm a human
        time.sleep(3)

        # Find username field
        username_input = driver.find_element_by_css_selector("input[name='username']")

        # Click on username field
        driver.execute_script("arguments[0].click();", username_input)

        # Send username
        username_input.send_keys(username)
        
        time.sleep(3)
        
        # Find password field
        try:
            password_input = driver.find_element_by_css_selector("input[name='password']")

        except:
            password_input = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/div/label/input')

        # Click on password field
        driver.execute_script("arguments[0].click();", password_input)
        
        # Send password
        password_input.send_keys(password)
        
        time.sleep(3)

        # Find and click log in button
        login_button = driver.find_element_by_xpath("//button[@type='submit']")
        driver.execute_script("arguments[0].click();", login_button)
    
        time.sleep(10)
        
        return driver, True


# Getting url of posts with hashtag

In [38]:
def scrape_links(hashtag, driver):
    """
    Scrape unique post links from instagram using Selenium
    Returns links of posts containing hashtag
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    
    hashtag_url = f"https://www.instagram.com/explore/tags/{hashtag}/"
    driver.get(hashtag_url)
    
    # Gets scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # List for unique instagram links
    unique_links = []
    
    # Loop until page ends
    while True:
        time.sleep(10)
        page_source = driver.page_source
        page_data = BeautifulSoup(page_source, "html.parser")
        data_body = page_data.find("body")
        
        for unique_link in data_body.findAll("a"):
            if re.match("/p", unique_link.get('href')):
                unique_links.append(f"https://www.instagram.com{unique_link.get('href')}")
            
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)
        
        # If new height equal to previous screen height, break because no more content
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(10)
        if last_height == new_height:
            break
        else:
            last_height = new_height
        
        # Update on scraping
        
        print (f"Scraped {len(unique_links)} links, {len(set(unique_links))} unique links")
        
    print(f"Finished scraping. Scraped {len(unique_links)} links, {len(set(unique_links))} unique links")
    print("\n")
    print("Closing driver.")
    driver.quit()
    unique_links = list(set(unique_links))
    return unique_links

In [257]:
driver = webdriver.Chrome('../util/chromedriver.exe')
driver, status = instagram_login(driver, "IS434_G1T5", "IS434@g1t5")
hashtag = "hawkerculturesg"
if status:
    unique_links = scrape_links(hashtag, driver)
else:
    print("Error logging in")

# Getting Caption from Post URL

In [1]:
def get_hashtags(driver, links):
    """
    Takes in instagram post URLs
    Returns hashtags in instagram post, seperated by comma
    """
    hashtags_list = []
    error_count = 0
    for index,link in enumerate(links):
        try:
            if index%10==0:
                print(index)
                try:
                    print(hashtags_list[-1])
                except:
                    pass

            driver.get(link)
            time.sleep(5)
            driver.refresh()
            time.sleep(10)
            page_source = driver.page_source
            raw_data = BeautifulSoup(page_source, "html.parser")
            data = raw_data.find_all("body")[0].find_all("script")[11]
            json_data = json.loads(data.text.strip().replace('window._sharedData =', '').replace(';', '')[:-1].split(",",1)[1])
            caption = json_data["items"][0]["caption"]["text"]
            words_from_post = caption.split()

            post_hashtags = []
            for word in words_from_post:
                if word[0] == "#":
                    post_hashtags.append(word)
            post_hashtags = ", ".join(post_hashtags)
            hashtags_list.append(post_hashtags)
            time.sleep(8)
        except:
            error_count+=1
            print(f"error at index: {index}")
            hashtags_list.append("Error")
    return hashtags_list

hashtags_list = get_hashtags(driver, unique_links) 

# Convert to dataframe and export as csv
hashtags_df = pd.DataFrame(list(zip(unique_links, hashtags_list)),
               columns = ['post_url', 'hashtags'])

output_path = "../data/instagram_hashtag_posts.csv"
hashtags_df.to_csv(output_path)
print(f"Output to {output_path}")